input: Drug, Duration, start/end date, and number of treatments 
output: excel sheet that can be used for data entry 

In [ ]:
import calendar
import datetime
import os
import itertools
import collections
import matplotlib.pyplot as plt
import numpy as np 
import statistics

# requires updating openpyxl to version 2.4.5 (pip install openpyxl --upgrade)
import openpyxl

In [ ]:
#creating named tuples that have properties useful for treatment type and mouse
Treatment = collections.namedtuple('Treatment', ('Name', 'StartDate', 'Duration', 'Mice'))
Mouse = collections.namedtuple('Mouse', ('Weight', 'TumorSize'))

In [ ]:
def master_function_generate_excel(input_excel, output_file, overwrite=False):
    #returns calendar of tasks and data sheet as an excel file 
    
    #create named tuples for each treatment and each drug (easier to organize)
    Treat = collections.namedtuple('Treat', ('Drugs'))
    Drug = collections.namedtuple('Drug', ('Name', 'Schedule'))
    
    #gather the raw data from the input excel file 
    raw_treatments, start_date, duration = gather_input(input_excel)
    date_list = make_date_list(start_date, duration)
    
    #create the data in an organized form in treatment list of treat tuples made of drug tuples
    treatments = [
        Treat([
            Drug(drug_name, drug_process(drug_name, start_date, duration))
            for drug_name in treatment
        ])
        for treatment in raw_treatments
    ] 

    #call output_schedule which will make the excel document with the calendar and data tables
    output_schedule(output_file, treatments, date_list, overwrite=overwrite)

In [ ]:
def gather_input(input_excel):
    #read in excel file and get really basic information
    wb = openpyxl.load_workbook(input_excel)
    sheet = wb.get_sheet_by_name('input')
    duration = sheet['A2'].value 
    start_date = sheet['B2'].value #stored as datetime!
    
    #treatments these mice will go through
    treatments = [
        [
            cell.value
            for cell in row[-2:]
            if cell.value != None
        ]
        for row in list(sheet.rows)[1:]
    ]
    
    return treatments, start_date, duration

In [ ]:
def make_date_list(start_date, duration):
    #make a list of all the days in the experiment
  
    base = start_date
    date_list = [base + datetime.timedelta(days=x) for x in range(0, duration)]
    return(date_list)


In [ ]:
def drug_process(drug, start_date, duration):
#which days in the calendar each drug needs to be given

    if drug is None:
        return None
    
    drug = drug.strip().lower()
    
    #these are the drugs
    if drug == 'olaprib':
        return [start_date + datetime.timedelta(days=x) for x in range(0, duration, 2)]
    if drug == 'cabozantib':
        return [start_date + datetime.timedelta(days=x) for x in range(0, duration, 3)]
    if drug == 'bmk177':
        return [start_date + datetime.timedelta(days=x) for x in range(0, duration, 7)]
    if drug == 'gsk135':
        return [start_date + datetime.timedelta(days=x) for x in range(0, duration, 7)]
    if drug == 'parpnib':
        return [start_date + datetime.timedelta(days=x) for x in range(0, duration, 2)]

In [ ]:
def output_schedule(output_filename, treatments, date_list, overwrite=False):
    #creates the output excel sheet with calendar and data table 
    
    #makes sure you aren't overwriting an existing file, or it deletes it 
    if not(overwrite) and os.path.isfile(output_filename):
        print('ERROR: file already exists. Please specify new filename.')
        return
    
    if overwrite and os.path.isfile(output_filename):
        os.remove(output_filename)
    
    wb = openpyxl.Workbook()
    
    tasks_sheet = wb.active
    
    #begining the task sheet 
    tasks_sheet.title = 'Tasks'
    
    date_to_row_lookup = {}
    
    #adding the dates
    for index, date in zip(range(len(date_list)), date_list):
        row = index + 2
        
        tasks_sheet.cell(
            column=1,
            row=row,
            value=date.date()
        )
        
        date_to_row_lookup[date] = row
    
    #generating the title of each treatment 
    for index, treatment in zip(range(len(treatments)), treatments):
        column = index + 2
        
        tasks_sheet.cell(
            row=1, 
            column=column,
            value=' + '.join([drug.Name for drug in treatment.Drugs])
        )
        
        #adds which drugs to treat when
        for drug in treatment.Drugs:
            for day in drug.Schedule:
                cell = tasks_sheet.cell(
                    column=column,
                    row=date_to_row_lookup[day]
                )
                
                if cell.value == None:
                    cell.value = 'Treat with {}'.format(drug.Name)
                else:
                    temp = cell.value 
                    cell.value = '{} and {}'.format(temp, drug.Name)
        
    #Adding titles and a space for each mouse measurement 
    for treatment in treatments:
        sheet_name = 'Treatment-{}'.format('+'.join([drug.Name for drug in treatment.Drugs]))
        ws = wb.create_sheet(sheet_name)
        
        ws['A1'] = 'Date'
        ws['B1'] = 'Mouse'
        ws['C1'] = 'Mouse Weight'
        ws['D1'] = 'Tumor Weight'
        
        row = 2
        for date in date_list: 
            for mouse in range(3): #3 mice / treatment 
                ws.cell(
                    column=1,
                    row=row,
                    value=date.date()
                )
                ws.cell(row=row, column=2, value='Mouse {}'.format(mouse+1))
                row = row + 1 
            
      
    wb.save(output_filename)
    wb.close()

In [ ]:
def get_treatments(input_excel):
    #get all the treatments from the input data, rather than input parameters
    
    wb = openpyxl.load_workbook(input_excel)
    
    sheets = wb.get_sheet_names()
    
    #all the things that start with Treatment- are treatments
    is_treatment = lambda s: s.startswith('Treatment-')
    
    treatment_sheets = filter(is_treatment, sheets)
    
    #specific named tuple for each data row with the important properties 
    raw_data_row = collections.namedtuple('RawDataRow', ('Date', 'Mouse', 'Weight', 'Tumor'))
    
    treatments = []
    
    #for each treatment gather the raw data 
    for sheet in treatment_sheets:
        ws = wb.get_sheet_by_name(sheet)
        name = sheet.replace('Treatment-', '', 1)
        
        #raw data only goes up to row 4
        raw_data = [
            raw_data_row(*[r.value for r in row[:4]])
            for row in list(ws.rows)[1:]
        ]
        
        #list of dates
        dates = [row.Date for row in raw_data]
        start_date = min(dates) #start date is the smallest 
        end_date = max(dates) #end date is the biggest
        duration = (end_date - start_date).days
        
        #gets the name of the mouse for sorting and grouping
        mouse_name_selector = lambda data: data.Mouse
        
        #sorting the data so can use groupby
        sorted_data = sorted(raw_data, key=mouse_name_selector)
        mice_group = itertools.groupby(sorted_data, mouse_name_selector)
        
        mice = []
        
        #sort by date, and get weight and tumor size and add to mouse object
        for key, group in mice_group:
            date_selector = lambda d: d.Date
            
            by_date = list(sorted(group, key=date_selector))
            
            weights = [data.Weight for data in by_date]
            tumors = [data.Tumor for data in by_date]
            
            mice.append(Mouse(weights, tumors))
            
        treatments.append(Treatment(name, start_date, duration, mice))
        
    return(treatments)

In [ ]:
def create_graphs_from_data(excel_doc):
    #generates the graphs from the given data
    treatments = get_treatments(excel_doc)
    
    generate_figures(treatments)

In [ ]:
def generate_figures(treatments):
    #generates the figures from the given data
    
    gen_fig_mouse_weight_over_time(treatments)
    gen_fig_tumorSize_over_time(treatments)

In [ ]:
def gen_fig_mouse_weight_over_time(treatments):
    #makes the figure for mouse weight over time
    
    plt.close()
    lines = []
    
    #each treatment is printed in a different color on the same plot
    for treatment in treatments:
        weights = [mouse.Weight for mouse in treatment.Mice]
        weights_per_time = zip(*weights)
        average_weights = list(map(statistics.mean, weights_per_time))
        line = plt.plot(range(len(average_weights)), average_weights, label=treatment.Name)
        
        lines.extend(line)
        
    plt.legend(handles=lines)
    
    ax = plt.subplot(111)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False) 
    ax.get_xaxis().tick_bottom()
    ax.set_xlabel('Days after first treatment')
    ax.set_ylabel('Mouse weight in grams')
    ax.get_yaxis().tick_left()
    
    plt.savefig('weight_overTime.png')
    plt.show()
    plt.close()

In [ ]:
def gen_fig_tumorSize_over_time(treatments):
    #makes the figure for the tumor size over time
    
    plt.close()
    lines = []
    
    for treatment in treatments:
        tumors = [mouse.TumorSize for mouse in treatment.Mice]
        tumor_per_time = zip(*tumors)
        average_tumor = list(map(statistics.mean, tumor_per_time))
        line = plt.plot(range(len(average_tumor)), average_tumor, label=treatment.Name)
        
        lines.extend(line)
        
    plt.legend(handles=lines)
    
   
    ax = plt.subplot(111)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False) 
    ax.get_xaxis().tick_bottom()
    ax.set_xlabel('Days after first treatment')
    ax.set_ylabel('Tumor weight in grams')
    ax.get_yaxis().tick_left()
    
    plt.savefig('Tumor_overTime.png')
    plt.show()
    plt.close()

In [ ]:
#example with test data 
master_function_generate_excel('input_example.xlsx', 'output.xlsx', overwrite=True)

In [ ]:
#example with test data
create_graphs_from_data('completed_output_example.xlsx')